In [65]:
import sys
import os
import torch
from pathlib import Path 
PYTHON_DIR = Path(os.getcwd()).parents[1].resolve()
sys.path.insert(0, PYTHON_DIR.as_posix()) if PYTHON_DIR.as_posix() not in sys.path else None
from rmsKit import rms_torch, lattice

In [81]:
mat_dir = Path("/Users/keisuke/Documents/projects/todo/worms/job/FF1D_loc/s_3_r_2_d_1_seed_3/1_mel")
H = np.load(mat_dir / "H/0.npy")
u = np.load(mat_dir /  "Adam/lr_0.001_epoch_1500/loss_0.2195750/u/0.npy")

In [89]:
U = np.kron(u, u)
Hu = U @ H @ U.T.conj()

In [90]:
np.round(np.abs(Hu),3)

array([[0.958, 0.061, 0.061, 0.061, 0.082, 0.077, 0.061, 0.077, 0.082],
       [0.061, 0.659, 0.034, 0.335, 0.317, 0.076, 0.031, 0.   , 0.012],
       [0.061, 0.034, 0.659, 0.031, 0.011, 0.   , 0.335, 0.077, 0.317],
       [0.061, 0.335, 0.031, 0.659, 0.317, 0.   , 0.034, 0.076, 0.012],
       [0.082, 0.317, 0.011, 0.317, 0.678, 0.061, 0.011, 0.061, 0.055],
       [0.077, 0.076, 0.   , 0.   , 0.061, 0.026, 0.077, 0.014, 0.061],
       [0.061, 0.031, 0.335, 0.034, 0.011, 0.077, 0.659, 0.   , 0.317],
       [0.077, 0.   , 0.077, 0.076, 0.061, 0.014, 0.   , 0.026, 0.061],
       [0.082, 0.012, 0.317, 0.012, 0.055, 0.061, 0.317, 0.061, 0.678]])

In [29]:
params = dict(
    sps = 3,
    rank = 2,
    dimension = 1,
    seed = 3,
    lt = 1
)

In [33]:
h, sps = lattice.FF.local(params)
h_torch = torch.tensor(h)

model = rms_torch.UnitaryRiemann(h.shape[1], sps, dtype=torch.float64)

optimizer = rms_torch.Adam(model.parameters(), lr=step, betas=(0.9, 0.999))

mel = rms_torch.MinimumEnergyLoss(h_torch, dtype = torch.float64)

optimizer 
step = 0.001
n_iter = 1000

for i in range(n_iter):
    optimizer.zero_grad()
    U = model.forward()
    loss = mel(U)
    loss.backward()
    model.update_riemannian_gradient()
    optimizer.step()
    print(f"loss : {loss.item()}")

loss : 0.17885808652022894
loss : 0.17779606543154025
loss : 0.1767302187410129
loss : 0.17566049504200842
loss : 0.1745868428365247
loss : 0.1735092107109386
loss : 0.17242754750850864
loss : 0.17134180249755349
loss : 0.1702519255342989
loss : 0.1691578672194829
loss : 0.16805957904786184
loss : 0.16695701354992343
loss : 0.16585012442517266
loss : 0.1647388666665055
loss : 0.16362319667529412
loss : 0.16250307236692096
loss : 0.16137845326663047
loss : 0.16024930059566822
loss : 0.15911557734779436
loss : 0.1579772483563453
loss : 0.15683428035212676
loss : 0.1556866420124845
loss : 0.15453430400199553
loss : 0.1533772390052437
loss : 0.15221542175225267
loss : 0.1510488290371228
loss : 0.14987743973050094
loss : 0.1487012347865193
loss : 0.14752019724482768
loss : 0.14633431222836957
loss : 0.14514356693756603
loss : 0.1439479506414969
loss : 0.14274745466674799
loss : 0.14154207238446315
loss : 0.14033179919623673
loss : 0.13911663251933049
loss : 0.13789657177178127
loss : 0.1366

In [63]:
import numpy as np
rg = (model.u[0].grad).clone().detach()
I = torch.eye(rg.shape[0])
rg = torch.kron(rg, I) + torch.kron(I, rg)
U = model.forward().clone().detach()

steps = np.arange(-10, 10, 0.1)
def landscpae_loss(step):
    Uc = torch.matrix_exp(-step * rg) @ U
    return mel(Uc).clone().detach()
loss_vals = []
for step in steps:
    loss_vals.append(landscpae_loss(step))

In [64]:
loss_vals

[tensor(0.1855, dtype=torch.float64),
 tensor(0.1840, dtype=torch.float64),
 tensor(0.1825, dtype=torch.float64),
 tensor(0.1810, dtype=torch.float64),
 tensor(0.1794, dtype=torch.float64),
 tensor(0.1779, dtype=torch.float64),
 tensor(0.1763, dtype=torch.float64),
 tensor(0.1748, dtype=torch.float64),
 tensor(0.1732, dtype=torch.float64),
 tensor(0.1716, dtype=torch.float64),
 tensor(0.1701, dtype=torch.float64),
 tensor(0.1685, dtype=torch.float64),
 tensor(0.1669, dtype=torch.float64),
 tensor(0.1653, dtype=torch.float64),
 tensor(0.1637, dtype=torch.float64),
 tensor(0.1621, dtype=torch.float64),
 tensor(0.1605, dtype=torch.float64),
 tensor(0.1589, dtype=torch.float64),
 tensor(0.1572, dtype=torch.float64),
 tensor(0.1556, dtype=torch.float64),
 tensor(0.1539, dtype=torch.float64),
 tensor(0.1523, dtype=torch.float64),
 tensor(0.1506, dtype=torch.float64),
 tensor(0.1490, dtype=torch.float64),
 tensor(0.1473, dtype=torch.float64),
 tensor(0.1456, dtype=torch.float64),
 tensor(0.14